In [1]:
import cv2
import numpy as np
import math
from PIL import Image
from random import shuffle
print(cv2.__version__)

4.10.0


In [2]:
import os
import torch
import matplotlib.pyplot as plt
from torch import nn
from torch.utils.data import DataLoader, ConcatDataset
from torchvision import transforms
from sklearn.model_selection import KFold
import winsound as sd

class SimpleConvNet(nn.Module):
  '''
    Simple Convolutional Neural Network
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Conv2d(1, 32, kernel_size=3,padding=1), # 1*300*300 -> 10*300*300
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #10*300*300 -> 10*150*150
      #풀링 추가
      nn.Conv2d(32, 64, kernel_size=3), #10*150*150 -> 20*148*148
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #20*148*148 -> 20*74*74
      #풀링 추가
      nn.Conv2d(64, 128, kernel_size=3), #20*74*74->40*72*72
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #40*72*72->40*36,36
      #풀링 추가
      nn.Conv2d(128, 256, kernel_size=3,padding=1), #40*36*36 -> 80*18*18
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #80*33*33->80*16*16
      #풀링 추가
      nn.Flatten(),
      nn.Linear(18 * 18 * 256, 50),
      nn.ReLU(),
      nn.Linear(50, 20),
      nn.ReLU(),
      nn.Linear(20, 2)
    )

  def forward(self, x):
    '''Forward pass'''
    return self.layers(x)

#280개 모델저장
models=[]
for i in range(280):
    model_load = torch.load("./loocv/last_model-LOOCV-{}.pth".format(i))
    models.append(SimpleConvNet())
    models[i].load_state_dict(model_load)


C:\Users\KimHojin\anaconda3\envs\py310\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] 지정된 프로시저를 찾을 수 없습니다'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [4]:
import cv2
import numpy as np

drawing = False
ix, iy = -1, -1
canvas = np.ones((350, 300, 3), dtype=np.uint8) * 255 #흰캔버스

def draw(event, x, y, flags, param):
    global ix, iy, drawing, result

    if event == cv2.EVENT_LBUTTONDOWN:
        if x <= 100 and y <= 50: #save 버튼 영역
            save_canvas()
            drawing = False
        elif x <= 200 and y <= 100: #clear 버튼 영역
            clear_canvas()
            result = 'NULL'
            drawing = False
        else:
            drawing = True
            ix, iy = x, y

    elif event == cv2.EVENT_MOUSEMOVE: #마우스따라다니는 선그리기
        if drawing:
            cv2.line(canvas, (ix, iy), (x, y), (0, 0, 0), 3)
            ix, iy = x, y

    elif event == cv2.EVENT_LBUTTONUP: 
        drawing = False
        cv2.line(canvas, (ix, iy), (x, y), (0, 0, 0), 3)
        ix,iy=-1,-1

def save_canvas():
    cv2.imwrite('drawing.png', canvas[50:]) #저장
    check_canvas() #검사

def clear_canvas():
    global canvas
    canvas = np.ones((350, 300, 3), dtype=np.uint8) * 255

def check_canvas():
    global result
    img = Image.open('drawing.png').convert('L')
    image = np.array([img])
    image=torch.FloatTensor([image])
    with torch.no_grad():
        #O라고 판단한 결과 개수 
        cnt=0
        #280개의 모델에서 전부 체크
        for model in models:
            outputs=model(image)
            #확률체크
            #m=nn.Softmax(dim=1)
            #print(m(outputs))
            _, predicted = torch.max(outputs.data, 1)
            if predicted.item()==0:
                cnt+=1
        print(cnt)
        if cnt>=140:
            result = "O"
        else:
            result = "X"

cv2.namedWindow('Paint')
cv2.setMouseCallback('Paint', draw)
result = 'NULL'

while True:
    #SAVE 버튼
    cv2.rectangle(canvas, (0, 0), (100, 49), (150, 150, 50), -1)
    cv2.putText(canvas, 'Save', (10, 30), cv2.FONT_ITALIC, 1, (0, 0, 0), 2)

    #Clear 버튼
    cv2.rectangle(canvas, (100, 0), (200, 49), (50, 150, 150), -1)
    cv2.putText(canvas, 'Clear', (110, 30), cv2.FONT_ITALIC, 1, (0, 0, 0), 2)

    #결과 츨력
    cv2.rectangle(canvas, (200, 0), (300, 49), (255, 255, 255), -1)
    cv2.putText(canvas, result, (210, 30), cv2.FONT_ITALIC, 1, (0, 0, 0), 2)

    cv2.imshow('Paint', canvas)

    if cv2.waitKey(1) & 0xFF == 27:  # ESC 키로 종료
        break
cv2.destroyAllWindows()


## 회고
    7월 1일~7월 25일 인공지능 스터디
    7월 1일부터 시작한 인공지능 스터디가 4주만에 끝이 났다.
## 1주차
    인공지능 스터디를 시작하기에 앞서 딥러닝 네트워크에 대해 각자 조사하고
    그중에서 cnn을 사용하여 이미지 분류를 하기로 결정하였다.
    작업환경은 colab을 사용한 파이썬환경에서 pytorch를 사용하여 구현하기로 하였다.
    분류할 이미지는 O , X 두 종류의 이미지로 분류하는 것을 목표로 하였다.
## 2주차
    먼저 MLP모델을 구현하고 이 모델을 응용하여 conv2d를 추가하여 cnn을 구현하였다. 
    k_fold를 통해 교차검증을 하여 모델의 정확도를 측정하였다.
    각자 구현한 모델을 발표하고 정확도를 올리기위한 향후 개선방향에 대하여 이야기하였다.
    cnn모델에서 활성함수와 손실함수등 기본 설정을 바꿔보면서 모델을 개선하기로 이야기하였다.
## 3주차
    2주차에서 이야기한 개선방향을 cnn에 적용하며 개선하였다.
    그리고 과적합이 발생하였는지 알아보기위해 각 실행에서 정확도와 손실을 알아보았다. 
    그 결과 10번정도의 실행으로 과적합이 발생하지않고 손실이 거의 0에 가까워 졌기에 실행 횟수는 10번으로 고정하였다. 
## 4주차
    학습한 모델에 opencv를 사용하여 직접그린 그림을 모델에 넣어 
    그린 그림이 O인지 X인지 판단하는 코드를 작성해 모델이 잘 학습하였는지 알아보았다.
## 후기
    처음해보는 스터디에 긴장을 많이 하였지만 스터디의 참여한 인원모두 적극적으로 참여해주어 긴장은 금방 해소되었다.
    스터디를 통해 인공지능의 개념에대해 서로 이해한 내용을 공유하며 잘못 이해한 내용을 서로 수정할 수 있었다.